In [3]:
import os
os.getcwd()

'/Users/dklaupaa/Desktop/chat_wrap_package'

# 1. Generate Synthetic Framework 

In [1]:
from mypythonpackage import FrameworkGenerator
import pandas as pd

/Users/dklaupaa/Library/Caches/pypoetry/virtualenvs/chat-wrap-package-h94m8WQz-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


this is just a test for my self to see what functions i'd actually like for it to have - realised that it is wayyy too specific, needs to be more modular still. ++++
needs to be flexible in the input it takes in like categories should could be defined in this notebook :) PLUS i want for it to not be overly csv dependent but to be worked with directly in here too

In [2]:

generator = FrameworkGenerator(model_name="llama-3.2-3b-instruct")


In [3]:
# Step 1 - Generate a framework
# the cool kinda thing here is that you can use any prompt you want and hence generate any framework you want !! :D 
df = generator.generate_framework(
    prompt_path="src/framework_generation/outline_prompts/prompt_default.py", ### THIS NEEDS TO BE UPDATED TO ALSO TAKE IN A DIRECTORY DIRECTLY 
    num_samples=10,
    api_url="http://localhost:1234/v1/completions"
    #json_out="output/data.json",
    #csv_out="output/data.csv"
)

print("Snippet of the generated framework:")
print(df.head())

Generating for category: Clarification
Skipping invalid output for category 'Clarification': 'choices'
Generating for category: Small Talk
Generating for category: Question
Number of duplicates removed: 5
Snippet of the generated framework:
                                                text       category
0  To clarify, you could rephrase the original qu...  Clarification
1  Can you rephrase or explain that in a differen...  Clarification
2  Can you explain the concept again, and provide...  Clarification
3  Can you rephrase the question or provide more ...  Clarification
4  Would you like me to break it down into smalle...  Clarification


In [12]:
# Step 2: Train + filter 
# to do the filtering a tiny labeled dataset is required for few shot learning, but that can fairly easily be made 
# Either from CSV
filtered_df = generator.filter_with_classifier(
    train_data="data/tiny_labeled_data.csv",
    synth_data="data/generated_tuning_data/FINAL_synthetic_combined500.csv",
    filtered_save_path="data/generated_tuning_data/PACKAKGE_QUALDATA.csv"
)

Max token: 34 Average token: 15.186046511627907


Map: 100%|██████████| 11/11 [00:00<00:00, 5193.89 examples/s]
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/dklaupaa/Library/Caches/pypoetry/virtualenvs/chat-wrap-package-h94m8WQz-py3.12/lib/python3.12/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.928502,0.545455,0.797980,0.545455,0.496503
2,No log,0.683839,1.000000,1.000000,1.000000,1.000000
3,No log,0.537540,1.000000,1.000000,1.000000,1.000000
4,No log,0.472429,1.000000,1.000000,1.000000,1.000000


Map: 100%|██████████| 850/850 [00:00<00:00, 14157.45 examples/s]


Filtered data saved to data/generated_tuning_data/PACKAKGE_QUALDATA.csv


In [5]:
# Or entirely in memory
df_train = "data/tiny_labeled_data.csv"
df_synth = df

filtered_df = generator.filter_with_classifier(
    train_data=df_train,
    synth_data=df_synth,
    classifier_model_name="distilbert-base-uncased"
)

Max token: 34 Average token: 15.186046511627907


Map: 100%|██████████| 11/11 [00:00<00:00, 4802.97 examples/s]
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/dklaupaa/Library/Caches/pypoetry/virtualenvs/chat-wrap-package-h94m8WQz-py3.12/lib/python3.12/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.928502,0.545455,0.797980,0.545455,0.496503
2,No log,0.683839,1.000000,1.000000,1.000000,1.000000
3,No log,0.537540,1.000000,1.000000,1.000000,1.000000
4,No log,0.472429,1.000000,1.000000,1.000000,1.000000


Map: 100%|██████████| 24/24 [00:00<00:00, 5878.15 examples/s]


In [6]:
# printing two rows
filtered_df.drop(columns=['label_id', 'predicted'], inplace=True)
unique_categories = filtered_df['category'].unique()
for category in unique_categories:
    print(filtered_df[filtered_df['category'] == category].head(2))

                                                text       category
0  To clarify, you could rephrase the original qu...  Clarification
1  Can you rephrase or explain that in a differen...  Clarification
                                          text    category
9   Do you have any fun plans for the weekend?  Small Talk
10                How's your day going so far?  Small Talk
                                                 text  category
19  What is the process by which plants convert su...  Question
20               What is the definition of terrorism?  Question


# 2. Generate Synthetic Tutor/Student Dialogue 
this is only for pof and probably not a part of the final pipeline 

In [1]:
from mypythonpackage import DialogueSimulator
from pathlib import Path

/Users/dklaupaa/Library/Caches/pypoetry/virtualenvs/chat-wrap-package-h94m8WQz-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#simulator = DialogueSimulator(backend="hf", model_id="gpt2")
simulator = DialogueSimulator(backend="mlx", model_id="mlx-community/Qwen2.5-7B-Instruct-1M-4bit")

Fetching 9 files: 100%|██████████| 9/9 [00:00<00:00, 79638.68it/s]


##### there still needs to be a loop to go over this like 10 times if you wanna mimick having an experiment with several kids 

In [3]:
df = simulator.simulate_dialogue(
    mode="evaluative_feedback",
    turns=6,
    log_dir=Path("logs/raw"),
    save_csv_path=Path("output/dialogue.csv")
)


--- Starting Dialogue Simulation ---

[Student]: Hi, I'm a student seeking assistance with my studies.


mx.metal.set_wired_limt is deprecated and will be removed in a future version. Use mx.set_wired_limit instead.
mx.metal.get_peak_memory is deprecated and will be removed in a future version. Use mx.get_peak_memory instead.
mx.metal.clear_cache is deprecated and will be removed in a future version. Use mx.clear_cache instead.


[Tutor]: Welcome! I'm here to help. Could you share the specific area or assignment you'd like assistance with?

Turn 2:
[Student]: Sure, here's my work for feedback:

"The importance of time management in student success cannot be overstated. Effective time management is crucial for academic success as it ensures that students have enough time to study, complete assignments, and participate in extra-curricular activities. Good time management also helps to reduce stress and improve overall well-being. A student who manages their time well can balance their responsibilities more effectively and achieve better results in their studies. "
[Tutor]: Your paragraph effectively communicates the importance of time management. To enhance it, consider adding examples of how time management strategies can be implemented. This could make your argument more concrete and relatable. How might you incorporate specific strategies or benefits into your explanation?

Turn 3:
[Student]: Thank you for the

# 3. Wrap Interactions Directly

In [7]:
from mypythonpackage import DialogueLogger

In [8]:
prompt = "You are an AI tutor for a student trying to solve a task. Your instructions are: " \
"Answer clearly and consicely, Check for understanding, Always be short and concise."

In [9]:
logger = DialogueLogger(
    api_url="http://127.0.0.1:1234/v1/chat/completions",
    model_name="llama-3.2-3b-instruct",
    temperature=0.7,
    context=prompt
)


df = logger.start_dialogue(participant_id="test_user2", output_path="data/logged_dialogue_data/")


Type 'quit' to exit and save the conversation.
Assistant: What specifically do you need help with in English (e.g. grammar, vocabulary, reading comprehension, writing?)?
Assistant: What are the sentences you're having trouble understanding? Share them with me, and I'll help clarify their meaning.
Assistant: This sentence is saying that a complex (likely a military base or housing complex) has homes for both married and single soldiers, as well as their families.
Assistant: Yes, I understand. The sentence is describing a type of housing that caters to both married and single individuals, specifically soldiers and their families.
Assistant: Here are a few more examples of complex housing for soldiers and their families:

* The base has modular homes for married couples and single airmen.
* The military housing complex offers apartments for families with children.
* The installation has townhouses and duplexes available for military personnel.

These examples illustrate the idea of provid

In [10]:
print(df)

   turn                                           user_msg  \
0     1                     hello i need help with english   
1     2                i need to understand some sentences   
2     3  The complex houses married and single soldiers...   
3     4                                  do you understand   
4     5                        do you know other examples?   
5     6  not housing but examples of complecated sentences   

                                           tutor_msg  
0  What specifically do you need help with in Eng...  
1  What are the sentences you're having trouble u...  
2  This sentence is saying that a complex (likely...  
3  Yes, I understand. The sentence is describing ...  
4  Here are a few more examples of complex housin...  
5  Here are a few more complex sentence examples:...  


# 4. Classification Time 
Now the dialogues are to be classified by a classifier trained on a relevant framework 

In [2]:
from mypythonpackage import PredictLabels

/Users/dklaupaa/Library/Caches/pypoetry/virtualenvs/chat-wrap-package-h94m8WQz-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# loading the model and data for prediction
predictor = PredictLabels(model_name="distilbert-base-uncased")

annotaded_df = predictor.run_pipeline(
    train_data="data/generated_tuning_data/PACKAKGE_QUALDATA.csv", 
    new_data="data/logged_dialogue_data/conversation_test_user2.csv", 
    # column names:
    text_column="text",
    label_column="category", 
    new_text_column="user_msg", # the text to be predicted
    prediction_save_path="data/final_outputs/first_annotated.csv"
)

Max token: 44 Average token: 15.587025316455696


Map:   0%|          | 0/632 [00:00<?, ? examples/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Map: 100%|██████████| 158/158 [00:00<00:00, 31536.12 examples/s]
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/dklaupaa/Library/Caches/pypoetry/virtualenvs/chat-wrap-package-h94m8WQz-py3.12/lib/python3.12/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and w

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.004372,1.000000,1.000000,1.000000,1.000000
2,No log,0.026333,0.993671,0.993914,0.993671,0.993702
3,No log,0.026607,0.993671,0.993914,0.993671,0.993702
4,No log,0.026102,0.993671,0.993914,0.993671,0.993702


Predicted data saved to data/final_outputs/first_annotated.csv


In [4]:
print(annotaded_df.head())

   turn                                           user_msg  \
0     1                     hello i need help with english   
1     2                i need to understand some sentences   
2     3  The complex houses married and single soldiers...   
3     4                                  do you understand   
4     5                        do you know other examples?   

                                           tutor_msg predicted_labels  
0  What specifically do you need help with in Eng...       Small Talk  
1  What are the sentences you're having trouble u...    Clarification  
2  This sentence is saying that a complex (likely...       Small Talk  
3  Yes, I understand. The sentence is describing ...    Clarification  
4  Here are a few more examples of complex housin...    Clarification  


okay laura conclusion for now the quality is terrible but the pipeline is working !!!!!!!! now data + framework just need to be fixed hmmm to actually be useful 